# タイトル: SageMaker Inference (Endpoint) で仮想のモデルをデプロイしよう🐱

■ 本ハンズオンのスコープ

* 以下のような 推論コードの理解
---
```
import os
def model_fn(model_dir):
    with open(os.path.join(model_dir,'my_model.txt')) as f:
        model = f.read()[:-1]
    return model
def predict_fn(input_data, model):
    response = f'{model} for the {input_data}st time'
    return response
```
---
* 以下のような SageMaker SDK のコードの理解
---
```
from sagemaker.pytorch import PyTorchModel

pytorch_model = PyTorchModel(
    name = model_name,
    model_data=model_s3_uri,
    role= role,
    framework_version = '1.11.0',
    py_version='py38',
    entry_point='inference.py',
    source_dir=f'./{source_dir}/'
)
pytorch_predictor = pytorch_model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large',
    enpoint_name=endpoint_name
)
```
---
* Inference endpoint の作成と呼び出しのハンズオン

■ 本ハンズオンのスコープ外
* Inference Endpoint 以外の SageMaker の機能
* 学習モデルの作成 (時間の関係上モデルの作成は行いません。)

# SageMaker Inference (Endpoint) - 座学編

## 全体のイメージ

![inference-overview](./images/inference-overview.png)

## 推論コードのお作法

![code-rule-inference](./images/code-rule-inference.png)

# SageMaker Inference (Endpoint) - ハンズオン編
[SageMaker PyTorch Container](https://github.com/aws/deep-learning-containers/tree/master/pytorch/inference/docker)  を使用します！


## 準備

### モジュールのインポート、定数の設定、boto3 クライアントの設定、ロールの取得

In [ ]:
import sagemaker
from typing import Final
from sagemaker.pytorch import PyTorchModel
from sagemaker.async_inference import AsyncInferenceConfig
import os, boto3, json, numpy as np
from io import BytesIO
from time import sleep
from uuid import uuid4
smr_client:Final = boto3.client('sagemaker-runtime')
sm_client:Final = boto3.client('sagemaker')
s3_client:Final = boto3.client('s3')
endpoint_inservice_waiter:Final = sm_client.get_waiter('endpoint_in_service')
role: Final[str] = sagemaker.get_execution_role()
region: Final[str] = sagemaker.Session().boto_region_name
bucket: Final[str] = sagemaker.Session().default_bucket()

### モデルと推論コードを保存するディレクトリを作成

In [ ]:
model_dir: Final[str] = 'model'
!if [ -d ./{model_dir} ]; then rm -rf ./{model_dir}/;fi
!mkdir ./{model_dir}/

source_dir: Final[str] = 'source'
!if [ -d ./{source_dir} ]; then rm -rf ./{source_dir}/;fi
!mkdir ./{source_dir}/

### モデル相当のテキストファイルを `tar.gz` で固めて S3 にアップロードする
* SageMaker で推論する場合は機械学習のモデルを `model.tar.gz` に固めておく必要がある
    * SageMaker Training を使ってモデルを保存した場合は自動で tar.gz になるが、このハンズオンでは Training Job を使わないため、手動で tar.gz に固める
    * 機械学習のモデルと言ったが、用意したファイルを読み込むコードを書き、その読み込んだデータを使って処理を行うだけなので必ずしも機械学習のモデルである必要はない
    * このチュートリアルでは Hello my great machine learning model と書かれたテキストファイル(`my_model.txt`)を作成して、`tar.gz` に固める
* `model.tar.gz` を推論環境で使うには予め S3 にアップロードしておく必要がある

#### `my_model.txt` 作成

In [ ]:
%%writefile ./{model_dir}/my_model.txt
Hello my great machine learning model

#### `my_model.txt` を `model.tar.gz` に固める

In [ ]:
%cd {model_dir}
!tar zcvf model.tar.gz ./*
%cd ..

#### `model.tar.gz` を S3 にアップロード

In [ ]:
model_s3_uri:Final[str] = sagemaker.session.Session().upload_data(
    f'./{model_dir}/model.tar.gz',
    key_prefix = 'hello_sagemaker_inference'
)
print(model_s3_uri)

### 推論コードを作成する
* 最低限 `model_fn` と `predict_fn` が必要
* `model_fn` は `model.tar.gz` に固めたモデルを読み込むコード
  * 第一引数に `model.tar.gz` を展開したディレクトリが入る
  * 返り値にモデルを返すと、`predict_fn` の第二引数に入れられる
* `predict_fn` は推論コード
  * 第一引数にリクエスト(推論したいデータ)が入る
  * 第二引数に model_fn の返り値が入る
  * 推論結果を返り値に入れる

In [ ]:
%%writefile ./{source_dir}/inference.py
import os
def model_fn(model_dir):
    with open(os.path.join(model_dir,'my_model.txt')) as f:
        model = f.read()[:-1] # 改行を除外
    return model
def predict_fn(input_data, model):
    response = f'{model} for the {input_data}st time'
    return response

## SageMaker SDK でモデルをデプロイしてリアルタイム推論
### SageMaker SDK を用いてモデルをデプロイ
SageMaker SDK でモデルをデプロイするには、[Model](https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#sagemaker.model.Model) クラスでモデルを定義する必要がある  
今回は AWS が管理・公開している PyTorch のコンテナを使うため、`Model` を継承した [PyTorchModel](https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/sagemaker.pytorch.html#sagemaker.pytorch.model.PyTorchModel) クラスを使用する。  
`PyTorchModel` では、モデルにつける任意の名前、使用するモデルの S3 の URI、フレームワークや Python のバージョン、推論コードなどを指定する。
PyTorchModel でインスタンスを生成したら、[deploy](https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#sagemaker.model.Model.deploy) メソッドでモデルをデプロイできる。デプロイ時はインスタンスタイプと台数、エンドポイントにつける任意の名前を設定する。

In [ ]:
# 名前の設定
model_name: Final[str] = 'PyTorchModel'
endpoint_name: Final[str] = model_name + 'Endpoint'

In [ ]:
# モデルとコンテナの指定
pytorch_model = PyTorchModel(
    name = model_name,
    model_data=model_s3_uri,
    role= role,
    framework_version = '1.11.0',
    py_version='py38',
    entry_point='inference.py',
    source_dir=f'./{source_dir}/'
)
pytorch_predictor = pytorch_model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large',
    enpoint_name=endpoint_name
)

In [ ]:
response = pytorch_predictor.predict(1)
print(response,type(response))

リクエストはできたがレスポンスがなぜか numpy array である。  
理由は [PyTorchPredictor](https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/sagemaker.pytorch.html#sagemaker.pytorch.model.PyTorchPredictor) に serializer , desirializer がデフォルトで設定されており、`predict` メソッドでエンドポイントにリクエストする前にリクエストデータ(↑の例では int 型の1)を numpy array に Serialize してリクエストし、レスポンスを受け取った後にレスポンスデータを numpy array に Desiarlize するため。

In [ ]:
print(pytorch_predictor.serializer, pytorch_predictor.deserializer)

In [ ]:
# endpointとモデルを削除
pytorch_predictor.delete_endpoint()
pytorch_model.delete_model()

## Boto3 でリアルタイム推論
serializer/desirializer は SageMaker SDK の機能で、推論エンドポイントに推論データをリクエストする環境(AWS Lambda など)には入っていないことが多い（boto3でやることが多い）。また、推論エンドポイント立ち上げもパイプラインに組み込む際は SageMaker SDKを使わない環境もありえる。  
一連の流れを Boto3 で実行してみてSerializer/Deserializerが無い場合の挙動を確認する。

### 推論コードを model.tar.gz に固めて S3 にアップロード
* pytorch の場合は推論コードを model.tar.gz に内包する必要がある
* SageMaker SDK では `deploy` メソッド実行時に裏側で推論コードを `model.tar.gz` に固めてアップロードしてくれていた
* boto3 でモデルをデプロイする場合は手動で `tar.gz` で固めて S3 にアップロードする必要がある

In [ ]:
%cd {model_dir}
!rm model.tar.gz
!cp ../{source_dir}/inference.py ./
!tar zcvf model.tar.gz ./*
%cd ..

In [ ]:
source_s3_uri:Final[str] = sagemaker.session.Session().upload_data(
    f'./{model_dir}/model.tar.gz',
    key_prefix = 'hello_sagemaker_inference'
)
print(source_s3_uri)

### EndpointConfigName 設定
SageMaker SDK では `deploy` メソッド実行時に自動で Model と同じ名前で EndpointConfig を作成するが、Boto3 は明示的に作成する必要がある。

In [ ]:
endpoint_config_name: Final[str] = model_name + 'EndpointConfig'

### モデル作成、エンドポイントコンフィグ作成、エンドポイント作成
1. [create_model](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_model) でモデルと推論環境（推論コードやコンテナイメージ、環境変数の設定）をパッケージ化した Model を作成する
2. [create_endpoint_config](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_endpoint_config) で使用する Model や推論に使うコンピューティングリソース（インスタンスタイプ、台数など）や負荷の配分を設定する
3. [create_endpoint](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_endpoint) で EndpointConfig で設定した内容をデプロイする

In [ ]:
# コンテナイメージの URI を取得
container_image_uri: Final[str] = sagemaker.image_uris.retrieve(
    "pytorch", 
    sagemaker.session.Session().boto_region_name, # ECR のリージョンを指定
    version='1.11.0', # SKLearn のバージョンを指定
    instance_type = 'ml.m5.large', # インスタンスタイプを指定
    image_scope = 'inference' # 推論コンテナを指定
)
print(container_image_uri)

create_endpoint は非同期 API で、すぐにレスポンスを返すが裏側ではエンドポイントを作成している。[endpoint_inservice_waiter.wait](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.get_waiter) でエンドポイント作成完了を待つことができる。（数分かかる）

In [ ]:
# Model 作成
response = sm_client.create_model(
    ModelName=model_name,
    PrimaryContainer={
        'Image': container_image_uri,
        'ModelDataUrl': model_s3_uri,
        'Environment': {
            'SAGEMAKER_CONTAINER_LOG_LEVEL': '20',
            'SAGEMAKER_PROGRAM': 'inference.py',
            'SAGEMAKER_REGION': region,
            'SAGEMAKER_SUBMIT_DIRECTORY': '/opt/ml/model/code'}
    },
    ExecutionRoleArn=role,
)
# EndpointConfig 作成
response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            'VariantName': 'AllTrafic',
            'ModelName': model_name,
            'InitialInstanceCount': 1,
            'InstanceType': 'ml.m5.large',
        },
    ],
)
# Endpoint 作成
response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name,
)
# Endpoint が有効化されるまで待つ
endpoint_inservice_waiter.wait(
    EndpointName=endpoint_name,
    WaiterConfig={'Delay': 5,}
)

一般的な`application/json`のヘッダでリクエストしてみる

In [ ]:
response = smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='application/json',
    Accept='application/json',
    Body='1'
)
predictions = json.loads(response['Body'].read().decode('utf-8'))
print(predictions)

なぜか torch tensor が表示されている。なぜ？  
[default_input_fn](https://github.com/aws/sagemaker-scikit-learn-container/blob/7773e19bf0df6bdd65f10076ff7e8ecc1390cb9b/src/sagemaker_sklearn_container/handler_service.py#L47) が影響している

In [ ]:
# default_input_fn 確認
!curl -s https://raw.githubusercontent.com/aws/deep-learning-containers/master/pytorch/inference/docker/build_artifacts/default_inference_handler.py | pygmentize

In [ ]:
# default_input_fn が呼ぶ decode の確認
!curl -s https://raw.githubusercontent.com/aws/sagemaker-inference-toolkit/master/src/sagemaker_inference/decoder.py | pygmentize

In [ ]:
# content_types の確認
!curl -s https://raw.githubusercontent.com/aws/sagemaker-inference-toolkit/master/src/sagemaker_inference/content_types.py | pygmentize

request header が `application/json` だったら `torch.FloatTensor(np.array(json.loads(input_data), dtype=None))` していたため、小数の torch tensor になっていた。  
上記を踏まえ、SageMaker SDK の [NumpySerializer](https://github.com/aws/sagemaker-python-sdk/blob/bd8ea409ae91b07ac148520f7631fba9feee0069/src/sagemaker/serializers.py#L148) の [_serialize_array()](https://github.com/aws/sagemaker-python-sdk/blob/bd8ea409ae91b07ac148520f7631fba9feee0069/src/sagemaker/serializers.py#L188) 相当を手動で行い、ヘッダ `application/x-npy` でリクエストしてみると、torch tensor にならずに済む

In [ ]:
buffer = BytesIO()
np.save(buffer,np.array(1))
response = smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='application/x-npy',
    Accept='application/json',
    Body=buffer.getvalue(),
)
predictions = json.loads(response['Body'].read().decode('utf-8'))
print(predictions)

### Model, EndpointConfig, Endpoint を削除

In [ ]:
sm_client.delete_endpoint(EndpointName=endpoint_name)
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm_client.delete_model(ModelName=model_name)

In [ ]:
# 削除が完了するまで待つ
sleep(5)

## 前処理と後処理の追加
### 前処理と後処理の関数作成
推論コードに `input_fn` と `output_fn` を記述すると、`default_input_fn` や `default_output_fn` は使われずにユーザーが作成した input_fn や output_fn が使われるようになる

In [ ]:
%%writefile ./{source_dir}/inference.py
import os, json
def model_fn(model_dir):
    with open(os.path.join(model_dir,'my_model.txt')) as f:
        hello = f.read()[:-1] # 改行を除外
    return hello
def input_fn(input_data, content_type):
    if content_type == 'text/csv':
        transformed_data = input_data.split(',')
    else:
        raise ValueError(f"Illegal content type {content_type}. The only allowed content_type is text/csv")
    print(input_data,transformed_data)
    return transformed_data
def predict_fn(transformed_data, model):
    prediction_list = []
    for data in transformed_data:
        if data[-1] == '1':
            ordinal = f'{data}st'
        elif data[-1] == '2':
            ordinal = f'{data}nd'
        elif data[-1] == '3':
            ordinal = f'{data}rd'
        else:
            ordinal = f'{data}th'
        prediction = f'{model} for the {ordinal} time'
        prediction_list.append(prediction)
    print(transformed_data,prediction_list)    
    return prediction_list
def output_fn(prediction_list, accept):
    if accept == 'text/csv':    
        response = ''
        for prediction in prediction_list:
            response += prediction + '\n'
        print(prediction_list,response)
    else:
        raise ValueError(f"Illegal accept type {accept}. The only allowed accept type is text/csv")
    return response, accept

### 前処理と後処理のコードをモデルと一緒に `model.tar.gz ` で固めて S3 にアップロード

In [ ]:
%cd {model_dir}
!rm model.tar.gz
!cp ../{source_dir}/inference.py ./
!tar zcvf model.tar.gz ./*
%cd ..

In [ ]:
source_s3_uri:Final[str] = sagemaker.session.Session().upload_data(
    f'./{model_dir}/model.tar.gz',
    key_prefix = 'hello_sagemaker_inference'
)
print(source_s3_uri)

### Model, EndpointConfig, Endpoint を作成

In [ ]:
# Model 作成
response = sm_client.create_model(
    ModelName=model_name,
    PrimaryContainer={
        'Image': container_image_uri,
        'ModelDataUrl': model_s3_uri,
        'Environment': {
            'SAGEMAKER_CONTAINER_LOG_LEVEL': '20',
            'SAGEMAKER_PROGRAM': 'inference.py',
            'SAGEMAKER_REGION': region,
            'SAGEMAKER_SUBMIT_DIRECTORY': '/opt/ml/model/code'}
    },
    ExecutionRoleArn=role,
)
# EndpointConfig 作成
response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            'VariantName': 'AllTrafic',
            'ModelName': model_name,
            'InitialInstanceCount': 1,
            'InstanceType': 'ml.m5.large',
        },
    ],
)
# Endpoint 作成
response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name,
)
# Endpoint が有効化されるまで待つ
endpoint_inservice_waiter.wait(
    EndpointName=endpoint_name,
    WaiterConfig={'Delay': 5,}
)

### 推論のユースケース
![usecase-inference](./images/usecase-inference.png)

### 推論

In [ ]:
response = smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='text/csv',
    Accept='text/csv',
    Body='1,2,3,10000'
)
predictions = response['Body'].read().decode('utf-8')
print(predictions)

### Model, EndpointConfig, Endpoint を削除

In [ ]:
sm_client.delete_endpoint(EndpointName=endpoint_name)
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm_client.delete_model(ModelName=model_name)

In [ ]:
sleep(5)

## 非同期推論
* 非同期推論は推論データを S3 に配置し、推論するときは S3 のどこに推論データがあるのかを引数に入れる
* 推論結果はレスポンスにある S3 の URI に格納されるが、レスポンスされたタイミングでは推論結果が置かれている保証はなく、処理が終わり次第配置される
* [create_endpoint_config](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_endpoint_config) の AsyncInferenceConfig 引数を設定することで非同期推論エンドポイントが立ち上がる

### エンドポイント作成

In [ ]:
# Model 作成
response = sm_client.create_model(
    ModelName=model_name,
    PrimaryContainer={
        'Image': container_image_uri,
        'ModelDataUrl': model_s3_uri,
        'Environment': {
            'SAGEMAKER_CONTAINER_LOG_LEVEL': '20',
            'SAGEMAKER_PROGRAM': 'inference.py',
            'SAGEMAKER_REGION': region,
            'SAGEMAKER_SUBMIT_DIRECTORY': '/opt/ml/model/code'}
    },
    ExecutionRoleArn=role,
)
# EndpointConfig 作成
response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            'VariantName': 'AllTrafic',
            'ModelName': model_name,
            'InitialInstanceCount': 1,
            'InstanceType': 'ml.m5.large',
        },
    ],
    AsyncInferenceConfig={
        "OutputConfig": {
            "S3OutputPath": f"s3://{bucket}/hello_sagemaker_inference/async_inference/output"
        },
    }
)
# Endpoint 作成
response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name,
)
# Endpoint が有効化されるまで待つ
endpoint_inservice_waiter.wait(
    EndpointName=endpoint_name,
    WaiterConfig={'Delay': 5,}
)


### 推論データ作成

In [ ]:
input_data: Final[str] = './input_data.csv'
with open(input_data,'wt') as f:
    f.write('2,3,4,1000')

### 推論データを S3 にアップロード

In [ ]:
input_data_s3_uri:Final[str] = sagemaker.Session().upload_data(
    './input_data.csv',
    key_prefix = 'hello_sagemaker_inference/async_inference'
)
print(input_data_s3_uri)

### 推論と推論結果を取得
推論には [invoke_endpoint_async](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker-runtime.html#SageMakerRuntime.Client.invoke_endpoint_async) を使う

In [ ]:
response = smr_client.invoke_endpoint_async(
    EndpointName=endpoint_name, 
    InputLocation=input_data_s3_uri,
    ContentType='text/csv',
    Accept='text/csv',
)
output_s3_uri = response['OutputLocation']
output_key = output_s3_uri.replace(f's3://{bucket}/','')
while True:
    result = s3_client.list_objects(Bucket=bucket, Prefix=output_key)
    exists = True if "Contents" in result else False
    if exists:
        print('!')
        obj = s3_client.get_object(Bucket=bucket, Key=output_key)
        predictions = obj['Body'].read().decode()
        print(predictions)
        break
    else:
        print('.',end='')
        sleep(0.1)

### リソース削除

In [ ]:
sm_client.delete_endpoint(EndpointName=endpoint_name)
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm_client.delete_model(ModelName=model_name)

In [ ]:
sleep(5)

## サーバーレス推論
* サーバーレス推論は、コンピューティングリソースをプロビジョンせず、推論が発生している時間に対して課金する推論方法
* サーバーレス推論の推論エンドポイントの立ち上げ方は [create_endpoint_config](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_endpoint_config) の Variant 内の ServerlessConfig で設定する

In [ ]:
# Model 作成
response = sm_client.create_model(
    ModelName=model_name,
    PrimaryContainer={
        'Image': container_image_uri,
        'ModelDataUrl': model_s3_uri,
        'Environment': {
            'SAGEMAKER_CONTAINER_LOG_LEVEL': '20',
            'SAGEMAKER_PROGRAM': 'inference.py',
            'SAGEMAKER_REGION': region,
            'SAGEMAKER_SUBMIT_DIRECTORY': '/opt/ml/model/code'}
    },
    ExecutionRoleArn=role,
)
# EndpointConfig 作成
response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            'ModelName': model_name,
            'VariantName': 'AllTrafic',
            'ServerlessConfig': { 
                'MemorySizeInMB': 1024, 
                'MaxConcurrency': 3
            }
        },
    ],
)
# Endpoint 作成
response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name,
)
# Endpoint が有効化されるまで待つ
endpoint_inservice_waiter.wait(
    EndpointName=endpoint_name,
    WaiterConfig={'Delay': 5,}
)

In [ ]:
response = smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='text/csv',
    Accept='text/csv',
    Body='1,2,3,10000'
)
predictions = response['Body'].read().decode('utf-8')
print(predictions)

In [ ]:
sm_client.delete_endpoint(EndpointName=endpoint_name)
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm_client.delete_model(ModelName=model_name)

In [ ]:
sleep(5)

## バッチ推論
* バッチ推論は溜まったデータをまとめて推論するコスト効率が良い方法で、レイテンシーを求められない時に使用する
* バッチ推論は [create_model](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_model) でモデルを作成したあと、create_transform_job でジョブを作成して推論を行う
* 推論結果は S3 に出力される

### 推論データを複数作成

In [ ]:
!mkdir -p batch
batch_data_dir: Final[str] = './batch'
input_data1: Final[str] = 'input_data1.csv'
input_data2: Final[str] = './input_data2.csv'
with open(os.path.join(batch_data_dir,input_data1),'wt') as f:
    f.write('3,4,5,100')
with open(os.path.join(batch_data_dir,input_data2),'wt') as f:
    f.write('9,8,7,6,5')
!aws s3 rm --recursive s3://{bucket}/{prefix}
prefix:Final[str] = 'hello_sagemaker_inference/transform_job'
input_prefix:Final[str] = prefix + '/input'
output_prefix:Final[str] = prefix + '/output'
input_data_s3_uri:Final[str] = sagemaker.Session().upload_data(batch_data_dir,key_prefix = input_prefix)
print(input_data_s3_uri)

### モデル作成

In [ ]:
# Model 作成
response = sm_client.create_model(
    ModelName=model_name,
    PrimaryContainer={
        'Image': container_image_uri,
        'ModelDataUrl': model_s3_uri,
        'Environment': {
            'SAGEMAKER_CONTAINER_LOG_LEVEL': '20',
            'SAGEMAKER_PROGRAM': 'inference.py',
            'SAGEMAKER_REGION': region,
            'SAGEMAKER_SUBMIT_DIRECTORY': '/opt/ml/model/code'}
    },
    ExecutionRoleArn=role,
)

### 推論ジョブ作成

In [ ]:
transform_job_name: Final[str] = f'{model_name}TransformJob-{uuid4()}'
print(transform_job_name)
response = sm_client.create_transform_job(
    TransformJobName=transform_job_name,
    ModelName=model_name,
    TransformInput={
        'DataSource': {
            'S3DataSource': {
                'S3DataType': 'S3Prefix',
                'S3Uri': f's3://{bucket}/{input_prefix}'
            }
        },
        'ContentType': 'text/csv',
    },
    TransformOutput={
        'S3OutputPath': f's3://{bucket}/{output_prefix}',
        'Accept': 'text/csv',
    },
    TransformResources={
        'InstanceType': 'ml.m5.large',
        'InstanceCount': 1,
    }
)

### 推論結果を取得

In [ ]:
while True:
    if sm_client.describe_transform_job(TransformJobName=transform_job_name)['TransformJobStatus'] == 'Completed':
        print('!')
        for content in s3_client.list_objects_v2(Bucket=bucket,Prefix=output_prefix)['Contents']:
            obj = s3_client.get_object(Bucket=bucket, Key=content['Key'])
            predictions = obj['Body'].read().decode()
            print(predictions)
        break
    else:
        print('.',end='')
        sleep(5)

## 全て削除

In [ ]:
# for endpoint in sm_client.list_endpoints()['Endpoints']:
#     response = sm_client.delete_endpoint(EndpointName=endpoint['EndpointName'])
#     print(response)

In [ ]:
# for endpoint_config in sm_client.list_endpoint_configs()['EndpointConfigs']:
#     response = sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config['EndpointConfigName'])
#     print(response)

In [ ]:
# for model in sm_client.list_models()['Models']:
#     response = sm_client.delete_model(ModelName=model['ModelName'])
#     print(response)